In [ ]:
import pandas as pd
import numpy as np
import ast
from unidecode import unidecode

In [ ]:
name_basics_url = 'https://datasets.imdbws.com/name.basics.tsv.gz'
name_basics_df = pd.read_csv(name_basics_url, compression='gzip', sep='\t', low_memory=False)

ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
ratings_df = pd.read_csv(ratings_url, compression='gzip', sep='\t', low_memory=False)

principals_url = "https://datasets.imdbws.com/title.principals.tsv.gz"
principals_df = pd.read_csv(principals_url, compression='gzip', sep='\t', low_memory=False)

crew_url = "https://datasets.imdbws.com/title.crew.tsv.gz"
crew_df = pd.read_csv(crew_url, compression='gzip', sep='\t', low_memory=False)

title_basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
title_basics_df = pd.read_csv(title_basics_url, compression='gzip', sep='\t', low_memory=False)

In [ ]:
# loading additional dataset to complete runtimeMinutes and genres columns \N values
additional_df = pd.read_csv("../Data/Raw/TMDB_movie_dataset_v11.tar.gz", compression='gzip', low_memory=False)

In [ ]:
name_basics_df

In [ ]:
ratings_df

In [ ]:
principals_df

In [ ]:
crew_df

In [ ]:
title_basics_df

In [ ]:
additional_df = additional_df.drop(columns=['7e92ba80a4301f38b207cdac3829114d.csv'])
additional_df

## [title_basics_df] to [movies_only_df] transformation:

In [ ]:
movies_only_df = title_basics_df[title_basics_df['titleType'] == 'movie'] # removing series and others from df - we only keep movies

In [ ]:
movies_only_df = movies_only_df.dropna() # remove 2 empty rows (the other datasets have no NANs) 

In [ ]:
movies_only_df = movies_only_df.drop(columns=['endYear'])

In [ ]:
# Replace "\N" with "missing_value" on genres, startYear and runetimeMinutes columns
columns_to_replace = ['runtimeMinutes', 'genres', "startYear"]
movies_only_df[columns_to_replace] = movies_only_df[columns_to_replace].replace("\\N", "missing_value")

# Also modifying genres column name to evade duplicates when adding data from additional_df
movies_only_df = movies_only_df.rename(columns={'genres': 'movieGenres'})

In [ ]:
movies_only_df = movies_only_df[movies_only_df['startYear'] != 'missing_value']
movies_only_df

## Filtering movies by startYear (1978-2025)

In [ ]:
print(movies_only_df["startYear"].value_counts())

In [ ]:
movies_only_df['startYear'] = movies_only_df['startYear'].astype(int) # transforming column to integer
movies_only_df = movies_only_df[movies_only_df["startYear"] >= 1978] # filtering with movies after 1978
movies_only_df = movies_only_df[movies_only_df["startYear"] < 2025] # taking off movies after 2025 too

In [ ]:
movies_only_df

In [ ]:
print(movies_only_df["startYear"].value_counts())

## Filling "\N" values with available values from additional dataset -> additional_df

In [ ]:
# Check movies_only_df
print("----------------genres---------------------")
print(movies_only_df["movieGenres"].value_counts())
print("")
print("------------runtimeMinutes-----------------")
print(movies_only_df["runtimeMinutes"].value_counts())
print("")
print("-------------------NaN---------------------")
print(movies_only_df.isna().sum())

In [ ]:
movies_only_df

In [ ]:
# Merge DataFrames to align data
merged_df = pd.merge(
    movies_only_df,
    additional_df,
    left_on='tconst',
    right_on='imdb_id',
    how='left'
)

# Print merged_df columns for debugging
print("Columns after merge:", merged_df.columns)

# Replace 'missing_value' in runtimeMinutes and movieGenres if available in additional data
merged_df['runtimeMinutes'] = merged_df.apply(
    lambda row: row['runtime'] if row['runtimeMinutes'] == 'missing_value' else row['runtimeMinutes'],
    axis=1
)
merged_df['movieGenres'] = merged_df.apply(
    lambda row: row['genres'] if row['movieGenres'] == 'missing_value' else row['movieGenres'],
    axis=1
)

# Recreate movies_cleaned_df with correct column references
movies_cleaned_df = merged_df[movies_only_df.columns.tolist()]

In [ ]:
# Check movies_cleaned_df -> we managed to reduce a bit the "missing_value"
# From 33972 "missing_value" in "movieGenres" to 30040 --> (missing_values are transformed to NaN on previous code cell)
# From 103212 in runtimeMinutes to 80830 --> will transform to integer and maybe delete the NaN and 0.0 rows
print("----------------genres---------------------")
print(movies_cleaned_df["movieGenres"].value_counts())
print("")
print("------------runtimeMinutes-----------------")
print(movies_cleaned_df["runtimeMinutes"].value_counts())
print("")
print("-------------------NaN---------------------")
print(movies_cleaned_df.isna().sum())

In [ ]:
movies_cleaned_final_df = movies_cleaned_df.dropna() # remove rows with empty values 
movies_cleaned_final_df['runtimeMinutes'] = movies_cleaned_final_df['runtimeMinutes'].astype(int) # transform runetimeMinutes values to integer
movies_cleaned_final_df = movies_cleaned_final_df[movies_cleaned_final_df['runtimeMinutes'] != 0] # drop rows where runetimeMinutes is 0

In [ ]:
movies_cleaned_final_df

In [ ]:
print("----------------genres---------------------")
print(movies_cleaned_final_df["movieGenres"].value_counts())
print("")
print("------------runtimeMinutes-----------------")
print(movies_cleaned_final_df["runtimeMinutes"].value_counts())
print("")
print("-------------------NaN---------------------")
print(movies_cleaned_final_df.isna().sum())

## Handing the runtimeMinutes outliers + applying movieGenres filters:

In [ ]:
# Count the number of entries where runtimeMinutes is greater than 200
count_above_200 = len(movies_cleaned_final_df[movies_cleaned_final_df['runtimeMinutes'] > 200])

# Alternatively, you can use sum() on a boolean condition
count_above_200_alternative = (movies_cleaned_final_df['runtimeMinutes'] > 200).sum()

# Display the result
print(f"Number of movies with runtimeMinutes greater than 200: {count_above_200}")

In [ ]:
# Function: Handling runtimeMinutes outliers 
def amend_runtime(runtime):
    # Setting a practical runtime range, adjust as per context
    MIN_RUNTIME = 60
    MAX_RUNTIME = 220
    
    if runtime < MIN_RUNTIME:
        return MIN_RUNTIME
    elif runtime > MAX_RUNTIME:
        return MAX_RUNTIME
    else:
        return runtime

# Apply the amendment function to correct the runtimeMinutes
movies_cleaned_final_df['runtimeMinutes'] = movies_cleaned_final_df['runtimeMinutes'].apply(amend_runtime)

In [ ]:
movies_cleaned_final_df = movies_cleaned_final_df.drop_duplicates(subset='tconst', keep='first') # removing duplicates
movies_cleaned_final_df['movieGenres'] = movies_cleaned_final_df['movieGenres'].apply(lambda x: x.split(',')[0])

In [ ]:
print("----------------genres---------------------")
print(movies_cleaned_final_df["movieGenres"].value_counts())
print("")
print("------------runtimeMinutes-----------------")
print(movies_cleaned_final_df["runtimeMinutes"].value_counts())
print("")
print("-------------------NaN---------------------")
print(movies_cleaned_final_df.isna().sum())

In [ ]:
# Define the genres you want to exclude
unwanted_genres = ['Documentary', 'Biography', 'Sport', 'History', 'Reality-TV', 'Talk-Show', 'News', 'TV Movie', 'Game-Show']

# Filter the DataFrame to exclude rows where 'movieGenres' is in unwanted_genres
movies_cleaned_final_df = movies_cleaned_final_df[~movies_cleaned_final_df['movieGenres'].isin(unwanted_genres)]

In [ ]:
movies_cleaned_final_df['movieGenres'] = movies_cleaned_final_df['movieGenres'].replace('Music', 'Musical') # Music genre -> Musical

In [ ]:
movies_cleaned_final_df

## Reducing ratings, principals and crew df to keep data from movies on movies_cleaned_final_df only:

In [ ]:
print(ratings_df.shape)
print(principals_df.shape)
print(crew_df.shape)

In [ ]:
# time to filter the other dataframes to keep only the movie related rows:
filtered_ratings_df = ratings_df.merge(movies_cleaned_final_df[['tconst']], on='tconst', how='inner')
filtered_principals_df = principals_df.merge(movies_cleaned_final_df[['tconst']], on='tconst', how='inner')
filtered_crew_df = crew_df.merge(movies_cleaned_final_df[['tconst']], on='tconst', how='inner')

In [ ]:
print(filtered_ratings_df.shape)
print(filtered_principals_df.shape)
print(filtered_crew_df.shape)

## Cleaning [filtered_ratings_df]

In [ ]:
filtered_ratings_df['averageRating'] = filtered_ratings_df['averageRating'].astype(float)
filtered_ratings_df['numVotes'] = filtered_ratings_df['numVotes'].astype(int)

In [ ]:
filtered_ratings_df

In [ ]:
print("-------------averageRating-----------------")
print(filtered_ratings_df["averageRating"].value_counts())
print("-----------------numVotes------------------")
print(filtered_ratings_df["numVotes"].value_counts())
print("")
print("-------------------NaN---------------------")
print(filtered_ratings_df.isna().sum())

In [ ]:
filtered_ratings_df.min()

In [ ]:
filtered_ratings_df.max()

In [ ]:
# Check for duplicates in 'tconst' and determine if there are any
duplicates_exist_ratings = filtered_ratings_df.duplicated(subset='tconst', keep=False).any()

# Output result
print("Are there any duplicates in the 'tconst' column?", duplicates_exist_ratings)

In [ ]:
filtered_ratings_df = filtered_ratings_df.drop_duplicates(subset='tconst', keep='first')

In [ ]:
filtered_ratings_df 

In [ ]:
print(filtered_ratings_df.isna().sum()) # no empty values, but the nb of rows is smaller than our movies sample

## Cleaning [filtered_principals_df]

In [ ]:
# filtered_principals_df = filtered_principals_df.drop(columns=['ordering'])

In [ ]:
filtered_principals_df

In [ ]:
print("-----------------category------------------")
print(filtered_principals_df["category"].value_counts())
print("-------------------job---------------------")
print(filtered_principals_df["job"].value_counts())
print("-------------------NaN---------------------")
print(filtered_principals_df.isna().sum())

## Cleaning [filtered_crew_df]

In [ ]:
filtered_crew_df

In [ ]:
print("-----------------directors-----------------")
print(filtered_crew_df["directors"].value_counts())
print("-----------------writers-------------------")
print(filtered_crew_df["writers"].value_counts())
print("-------------------NaN---------------------")
print(filtered_crew_df.isna().sum())

In [ ]:
filtered_crew_df['directors'] = filtered_crew_df['directors'].apply(lambda x: x.split(',')[0])
filtered_crew_df['writers'] = filtered_crew_df['writers'].apply(lambda x: x.split(',')[0])

In [ ]:
filtered_crew_df = filtered_crew_df.drop_duplicates(subset='tconst', keep='first')
filtered_crew_df

In [ ]:
# adding the director names from "name_basics_df"
final_crew_df = pd.merge(
    filtered_crew_df,
    name_basics_df,
    left_on='directors',
    right_on='nconst',
    how='left'
)

# moving the "primaryName" to "directors"
final_crew_df['directors'] = final_crew_df['primaryName'] 

# keeping the initial columns
final_crew_df = final_crew_df[filtered_crew_df.columns.tolist()]

In [ ]:
# adding the writer names from "name_basics_df"
final_crew_df = pd.merge(
    final_crew_df,
    name_basics_df,
    left_on='writers',
    right_on='nconst',
    how='left'
)

# moving the "primaryName" to "directors"
final_crew_df['writers'] = final_crew_df['primaryName'] 

# keeping the initial columns
final_crew_df = final_crew_df[filtered_crew_df.columns.tolist()]

In [ ]:
final_crew_df

In [ ]:
print(final_crew_df.isna().sum()) # we are missing some directors and many writers

In [ ]:
# drop writers column
final_crew_df = final_crew_df.drop(columns=['writers'])

In [ ]:
print(final_crew_df.isna().sum())

## Merging [final_crew_df], [filtered_ratings_df] and [movies_cleaned_final_df]

In [ ]:
movies_cleaned_final_df

In [ ]:
# Merge based on 'tconst' (2 steps) - 1st step:
merged_df_1 = pd.merge(movies_cleaned_final_df, final_crew_df, on='tconst', how='left')

# 2nd step
final_movies_df = pd.merge(merged_df_1, filtered_ratings_df, on='tconst', how='left')

In [ ]:
final_movies_df

In [ ]:
# creating a new DF removing the empty values to have all the movies with ratings
ratings_movies_df = final_movies_df.dropna()

In [ ]:
ratings_movies_df['averageRating'] = ratings_movies_df['averageRating'].astype(float)
ratings_movies_df['numVotes'] = ratings_movies_df['numVotes'].astype(int)
ratings_movies_df

In [ ]:
print("-----------------isAdult-----------------")
print(ratings_movies_df["isAdult"].value_counts()) # The isAdult values from the dataset are wrong so we are not using them for analytical purposes
print("---------...----numVotes-----------------")
print(ratings_movies_df["numVotes"].value_counts())
print("-------------averageRating---------------")
print(ratings_movies_df["averageRating"].value_counts())

# Actors and movie-ratings DataFrame

In [ ]:
# ratings_movies_df
# name_basics_df
# filtered_principals_df

In [ ]:
actors_filtered_df = filtered_principals_df[filtered_principals_df['category'].isin(['actor', 'actress'])]

In [ ]:
actors_filtered_df

In [ ]:
# Merge based on 'tconst' (2 steps) - 1st step:
merged_df_2 = pd.merge(actors_filtered_df, ratings_movies_df, on='tconst', how='left')

# 2nd step
actors_df = pd.merge(merged_df_2, name_basics_df, on='nconst', how='left')

In [ ]:
actors_df = actors_df.dropna()
actors_df = actors_df.drop(columns=['titleType', "job", "knownForTitles", "primaryProfession", "primaryProfession"])

In [ ]:
actors_df.columns

In [ ]:
actors_df = actors_df.rename(columns={'startYear': 'MovieReleaseYear'})

new_column_order = ["tconst","nconst",'primaryName',"category", "birthYear", "deathYear", "primaryTitle", 'MovieReleaseYear', "isAdult", "averageRating", "numVotes"] + [col for col in actors_df.columns if col not in ["tconst","nconst",'primaryName',"category", "birthYear", "deathYear", "primaryTitle", 'MovieReleaseYear', "isAdult", "averageRating", "numVotes"]]
actors_df = actors_df[new_column_order]

In [ ]:
# amending types
actors_df['averageRating'] = actors_df['averageRating'].astype(float)
actors_df['numVotes'] = actors_df['numVotes'].astype(int)
actors_df['MovieReleaseYear'] = actors_df['MovieReleaseYear'].astype(int)
actors_df['runtimeMinutes'] = actors_df['runtimeMinutes'].astype(int)

In [ ]:
actors_df = actors_df[actors_df["MovieReleaseYear"] >= 2010] # filtering actors with movies after 2010 because the dataset is huge

In [ ]:
actors_df

In [ ]:
print(actors_df.isna().sum()) # no nulls, only \N wich is OK

# Preparing SQL Tables

In [ ]:
# merging movies - crew - ratings:
merged_df_sql1 = pd.merge(movies_cleaned_final_df, filtered_crew_df, on='tconst', how='left')

movies_sql = pd.merge(merged_df_sql1, filtered_ratings_df, on='tconst', how='left')

## 1st: characters table


In [ ]:
characters_sql = filtered_principals_df[filtered_principals_df['category'].isin(['actor', 'actress'])] # filter by actor/actress

# column order:
characters_column_order = ["ordering","characters"] + [col for col in characters_sql.columns if col not in ["ordering","characters"]]
characters_sql = characters_sql[characters_column_order]

characters_sql['ordering'] = range(1, len(characters_sql) + 1) # preparing ordering as main key with unique values

characters_sql['characters'] = characters_sql['characters'].apply(lambda x: ast.literal_eval(x)[0] if x != '\\N' else x) # cleaning column characters

characters_sql = characters_sql.drop(columns=['job']) # drop column "job" as all values are \N

In [ ]:
# Merge characters_sql with movies_sql to include the startYear column
final_characters_sql = characters_sql.merge(movies_sql[['tconst', 'startYear']], on='tconst', how='left')

final_characters_sql = final_characters_sql[final_characters_sql["startYear"] >= 2020] # reducing the amount of data

final_characters_sql = final_characters_sql.drop(columns=['startYear']) # dropping startYear after filtering

In [ ]:
final_characters_sql

## 2nd: crew table

In [ ]:
movies_sql['averageRating'] = movies_sql['averageRating'].astype(float)

# Convert 'numVotes' to integer using a fill value, such as 0 for NaN
movies_sql['numVotes'] = movies_sql['numVotes'].fillna(0).astype(int)

In [ ]:
crew_sql = movies_sql[["tconst", 'directors', 'writers']]
crew_sql.insert(0, 'id', range(1, len(crew_sql) + 1))
crew_sql = crew_sql.drop(columns=['writers'])

In [ ]:
# Merge crew_sql with movies_sql to include the startYear column
final_crew_sql = crew_sql.merge(movies_sql[['tconst', 'startYear']], on='tconst', how='left')

# Filter final_crew_sql to include only rows where startYear is 2020 or later
final_crew_sql = final_crew_sql[final_crew_sql["startYear"] >= 2020]

final_crew_sql = final_crew_sql.drop(columns=['startYear']) # dropping startYear after filtering

# final_crew_sql['directors'] = final_crew_sql['directors'].apply(lambda x: x.split(',')[0])

In [ ]:
final_crew_sql

## 3rd:  movies table

In [ ]:
movies_sql = movies_sql.drop(columns=['directors', 'writers'])

In [ ]:
# cleaning special characters from title columns:

# Apply unidecode to all string columns to remove accents
movies_sql = movies_sql.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)

In [ ]:
movies_sql['averageRating'] = movies_sql['averageRating'].apply(lambda x: None if pd.isna(x) else x)
movies_sql['averageRating'] = movies_sql['averageRating'].fillna(-1.0)

In [ ]:
final_movies_sql = movies_sql[movies_sql["startYear"] >= 2020] # reducing the amount of data
final_movies_sql = final_movies_sql.drop(columns=['isAdult']) # dropping startYear after filtering
final_movies_sql

In [ ]:
print("-------------startYear---------------")
print(movies_sql["startYear"].dtype)
print("-------------averageRating---------------")
print(movies_sql["averageRating"].dtype)

# 4th: Individuals table

In [ ]:
individuals_sql = name_basics_df.applymap(lambda x: unidecode(x) if isinstance(x, str) else x) # Apply unidecode to all string columns to remove accents

In [ ]:
# Identify unique nconst values in final_characters_sql and final_crew_sql (for directors)
nconst_chars = final_characters_sql['nconst'].unique()
nconst_crew_directors = final_crew_sql['directors'].unique()

# Compute the intersection of these two sets
common_nconst = set(nconst_chars) & set(nconst_crew_directors)

In [ ]:
final_individuals_sql = individuals_sql[individuals_sql['nconst'].isin(common_nconst)]

In [ ]:
# Replace '\N' with 'unknown' in 'birthYear' and 'alive' in 'deathYear'
final_individuals_sql['birthYear'] = final_individuals_sql['birthYear'].replace('\\N', 'unknown')
final_individuals_sql['deathYear'] = final_individuals_sql['deathYear'].replace('\\N', 'alive')

In [ ]:
final_individuals_sql['knownForTitles'] = final_individuals_sql['knownForTitles'].apply(lambda x: x.split(',')[0])

In [ ]:
final_individuals_sql

# Creating streamlit tables

In [ ]:
actors_app = actors_df.drop(columns=['tconst', 'nconst', 'isAdult', 'ordering', 'directors', 'originalTitle', 'runtimeMinutes', 'originalTitle']) 
actors_app = actors_app.rename(columns={'MovieReleaseYear': 'Movie_Release_Year', 'primaryName': 'Name', 'category': 'Gender', 'primaryTitle': 'Movie_Title', 
                                        'birthYear': 'Birth_Year', 'deathYear': 'Death_Year', 'averageRating': 'Average_Rating', 'numVotes': 'Votes',
                                        'characters': 'Character', 'movieGenres': 'Movie_Genre'})
actors_app = actors_app[actors_app["Movie_Release_Year"] >= 2010]
actors_app

In [ ]:
movies_app = ratings_movies_df.drop(columns=['tconst', 'titleType', 'isAdult']) 
movies_app = movies_app.rename(columns={'primaryTitle': 'Title', 'originalTitle': 'Original_Title', 'startYear': 'Movie_Release_Year', 'runtimeMinutes': 'Movie_Duration_(mins.)', 
                                        'movieGenres': 'Movie_Genre', 'directors': 'Director', 'averageRating': 'Average_Rating', 'numVotes': 'Votes'})
movies_app = movies_app[movies_app["Movie_Release_Year"] >= 2010]
movies_app

# Generating .csv files

In [ ]:
# EDA and analysis DataFrames --> compressed :)
actors_df.to_csv('../Data/Clean/actors.csv.gz', sep=';', encoding='utf-8', index=False, compression='gzip')
ratings_movies_df.to_csv('../Data/Clean/movies.csv.gz', sep=';', encoding='utf-8', index=False, compression='gzip')

In [ ]:
#tableau .csv
actors_df.to_csv('../Data/Clean/actors.csv', sep=';', encoding='utf-8', index=False)
ratings_movies_df.to_csv('../Data/Clean/movies.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
# SQL DataFrames --> separator = ; / encoding = utf8 / index = false
# SQL - remember not restrict as not null on sql!
final_characters_sql.to_csv('../Data/Clean/characters_sql.csv', sep=';', encoding='utf-8', index=False)
final_movies_sql.to_csv('../Data/Clean/movies_sql.csv', sep=';', encoding='utf-8', index=False)
final_crew_sql.to_csv('../Data/Clean/crew_sql.csv', sep=';', encoding='utf-8', index=False)
final_individuals_sql.to_csv('../Data/Clean/individuals_sql.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
# Streamlit files:
movies_app.to_csv('../Data/Clean/movies_app.csv', sep=';', encoding='utf-8', index=False)
actors_app.to_csv('../Data/Clean/actors_app.csv', sep=';', encoding='utf-8', index=False)